In [2]:
import requests
import pandas as pd
import my_key as key
import csv
import json

In [19]:
api_key = key.api_key

In [4]:
df = pd.read_csv("gps_data.csv")
df.insert(0, 'id', range(len(df)))
df.head()


,id,Ville,Latitude,Longitude
0,0,Mont Saint Michel,48.635954,-1.511460
1,1,Saint Malo,48.649518,-2.026041
2,2,Bayeux,49.276462,-0.702474
3,3,Le Havre,49.493898,0.107973
4,4,Rouen,49.440459,1.093966


In [ ]:
# Fonction pour récupérer la température, l'humidité et le champ "main" d'une ville
def get_temperature_humidity_and_main(latitude, longitude):
    url = f"https://api.openweathermap.org/data/3.0/onecall?lat={latitude}&lon={longitude}&appid={api_key}"
    response = requests.get(url)
    data = response.json()
    if "current" in data:
        temperature_celsius = data["current"]["temp"] - 273.15  # Convertir de Kelvin en Celsius
        humidity = data["current"]["humidity"]
        weather_main = data["current"]["weather"][0]["main"]
        return temperature_celsius, humidity, weather_main
    else:
        return None, None, None

In [ ]:
csv_filename = "meteo_data.csv"

# Créer un fichier CSV
with open('meteo_data.csv', 'w', newline='') as csvfile:
    fieldnames = ['Ville', 'Température (°C)', 'Humidité (%)', 'Météo principale']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    # Parcourir les lignes du DataFrame
    for index, row in df.iterrows():
        ville = row['Ville']
        latitude = row['Latitude']
        longitude = row['Longitude']
        temperature, humidity, weather_main = get_temperature_humidity_and_main(latitude, longitude)
        if temperature is not None:
            temperature_rounded = round(temperature)  # Arrondir la température à l'entier le plus proche
            writer.writerow({'Ville': ville, 'Température (°C)': temperature_rounded, 'Humidité (%)': humidity, 'Météo principale': weather_main})
            print(f"Ville: {ville}, Température : {temperature_rounded}°C, Humidité : {humidity}%, Météo principale: {weather_main}")
        else:
            writer.writerow({'Ville': ville, 'Température (°C)': 'N/A', 'Humidité (%)': 'N/A', 'Météo principale': 'N/A'})

print("Les résultats ont été enregistrés dans 'meteo_data.csv'")

In [5]:
df_meteo = pd.read_csv("meteo_data.csv")

In [8]:
df_meteo.insert(0, 'id', range(len(df)))
df_meteo.head()

,id,ville,temperature (°C),humidite (%),meteo principale
0,0,Mont Saint Michel,15,87,Clouds
1,1,Saint Malo,14,82,Clouds
2,2,Bayeux,14,88,Clouds
3,3,Le Havre,14,88,Clouds
4,4,Rouen,14,85,Clouds


In [13]:
df_meteo.rename(columns={'temperature (°C)': 'temperature', 'humidite (%)': 'humidite'}, inplace=True)
df_meteo.head()

,id,ville,temperature,humidite,meteo principale
0,0,Mont Saint Michel,15,87,Clouds
1,1,Saint Malo,14,82,Clouds
2,2,Bayeux,14,88,Clouds
3,3,Le Havre,14,88,Clouds
4,4,Rouen,14,85,Clouds


Création d'un seul fichier csv

In [3]:
with open('booking.json') as f:
    data_json = json.load(f)

df_json = pd.DataFrame(data_json)

In [6]:
df_fusionne = df_meteo.merge(df_json, on='ville', how='left')


In [7]:
new_col_names = {
    'ville': 'city',
    'temperature (°C)': 'temperature',
    'humidite (%)': 'humidity',
    'meteo principale': 'weather',
    'name': 'hotel'
}


In [8]:
df_fusionne = df_fusionne.rename(columns=new_col_names)


In [12]:
df_final = df_fusionne[['city', 'temperature', 'humidity', 'weather', 'hotel']]


In [14]:
df_final.to_csv('fichier_final.csv', index=True, index_label='id', columns=['city', 'temperature', 'humidity', 'weather', 'hotel'])


In [11]:
df_fusionne.head()

,city,temperature,humidity,weather,hotel,website,score,desc,lat,lon
0,Mont Saint Michel,15,87,Clouds,NaN,NaN,NaN,NaN,NaN,NaN
1,Saint Malo,14,82,Clouds,NaN,NaN,NaN,NaN,NaN,NaN
2,Bayeux,14,88,Clouds,NaN,NaN,NaN,NaN,NaN,NaN
3,Le Havre,14,88,Clouds,NaN,NaN,NaN,NaN,NaN,NaN
4,Rouen,14,85,Clouds,NaN,NaN,NaN,NaN,NaN,NaN


Top 5 weather localization

In [ ]:
csv_filename = "top_five_weather.csv"

mask = (df_meteo["meteo principale"] == "clear sky") | (df_meteo["meteo principale"] == "light rain")
df_top_five = df_meteo.sort_values(by=['temperature'], ascending=False)[mask].head(5)

# Créer un fichier CSV
with open(csv_filename, 'w', newline='') as csvfile:
    fieldnames = ['Ville', 'Température (°C)', 'Météo principale']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    # Parcourir les lignes du DataFrame
    for index, row in df_top_five.iterrows():
        ville = row['ville']
        temperature = row['temperature']
        weather = row['meteo principale']
        writer.writerow({'Ville': ville, 'Température (°C)': temperature, 'Météo principale': weather})
        print(f"Ville: {ville}, Température : {temperature}°C, Météo principale: {weather}")
   

print("Les résultats ont été enregistrés dans 'meteo_data.csv'")